In [2]:
import pandas as ps
import numpy as np
from trueskill import Rating, quality_1vs1, rate_1vs1
import globalConstants as gConst
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from copy import deepcopy
import seaborn as sn
import import_ipynb
import mTurkAnalyzer 

importing Jupyter notebook from mTurkAnalyzer.ipynb


In [3]:
class BiasEstimator:
    def __init__(self,recordClass,tsaAbs,tsaRel,errDict={},biasDict={}):
        self.recordAnalyzer = recordClass
        self.tsaAbs = tsaAbs
        self.tsaRel = tsaRel
        self.errDict = errDict
        self.biasDict = biasDict
        self.URBAN_MAP = {0:'Seattle',1:'Urban',2:'Suburb'}
        self.VIEWING_ANGLES = ['straight','side']
        self.meanAvgDict = {}
    
    def predictVote(self,a,b,strength,label):
        RatingA = Rating(float(a[strength + '_mu_' + label]),float(a[strength + '_sigma_' + label]))
        RatingB = Rating(float(b[strength + '_mu_' + label]),float(b[strength + '_sigma_' + label]))
        pTie = quality_1vs1(RatingA,RatingB)
        intensity = (1-pTie)*50
        vote = 50+ intensity if RatingB.mu > RatingA.mu else 50-intensity
        return vote
    
    def getPredictions(self,a,b,label):
        predictions = []
        for strength in ['slight','mod','strong']:
            predictions.append(self.predictVote(a,b,strength,label))
        # calc avg prediction
        predictions.append(sum(predictions)/3.0) 
        return(predictions)
    
    def calcErrors(self,predictions,actual):
        err = []
        for pred in predictions:
            err.append(pred - actual)
        return err
    
    def calcBiases(self,error):
        biasLeft = -1*error
        biasRight = error
        return [biasLeft,biasRight]
    
    def calcUrbanCat(self,img1Cat,img2Cat):
        #print("image 1 cat: %i, image2 cat: %i" % (img1Cat,img2Cat))
        if(int(img1Cat)==int(img2Cat)):
            return gConst.URBAN_CATEGORIES[int(img1Cat)]
        else:
            return gConst.URBAN_CATEGORIES[len(gConst.URBAN_CATEGORIES)-1]
    
    def getErrs(self,record,label,absVal=True):
        lImg = record['l_img_' + label]
        rImg = record['r_img_' + label]
        if(absVal):
            lTS = self.tsaAbs.getTSOneImageOneLabel(lImg,label)
            rTS = self.tsaAbs.getTSOneImageOneLabel(rImg,label)
        else:
            lTS = self.tsaRel.getTSOneImageOneLabel(lImg,label)
            rTS = self.tsaRel.getTSOneImageOneLabel(rImg,label)
        predictions = self.getPredictions(lTS,rTS,label)
        varName = 'abs_cont_outcome_' + label if absVal else 'rel_cont_outcome_' + label
        actual = record[varName]
        binaryErr,catErr = self.calcErrors([predictions[0],predictions[3]],actual)
        lBias,rBias = self.calcBiases(catErr)
        lUrb = lTS['urban_cat']
        rUrb = rTS['urban_cat']
        urbanCat = self.calcUrbanCat(lUrb,rUrb)
        
        outcomeDict = {
            'binaryErr':binaryErr,
            'catErr':catErr,
            'compare_cat':urbanCat,
            'region':record['mt_region'],
            'lBias':lBias,
            'rBias':rBias,
            'lImg':lImg,
            'rImg':rImg,
            'lUrb':lUrb,
            'rUrb':rUrb
        }
        return(outcomeDict)
    
    def calcErrsForLabel(self,label,maxSample=999999,absVal=True):
        binaryErr, catErr,compareCat,mtCat,lBias,rBias,lImg,rImg,lUrb,rUrb = [[] for _ in range(10)]
        biasDF = ps.DataFrame()
        numRecords = self.recordAnalyzer.getNumRecords(allRecords=False)
        for index in range(min(numRecords,maxSample)):
            tempRecord = self.recordAnalyzer.getRecordAtIndex(index,allRecords=False)
            errResults = self.getErrs(tempRecord,label,absVal)    
            binaryErr.append(errResults['binaryErr'])
            catErr.append(errResults['catErr'])
            compareCat.append(errResults['compare_cat'])
            mtCat.append(errResults['region'])
            lBias.append(errResults['lBias'])
            rBias.append(errResults['rBias'])
            lImg.append(errResults['lImg'])
            rImg.append(errResults['rImg'])
            lUrb.append(errResults['lUrb'])
            rUrb.append(errResults['rUrb'])
        labelErrs = ps.DataFrame()
        labelErrs['binaryErr'] = binaryErr
        labelErrs['catErr'] = catErr
        labelErrs['compareCat'] = compareCat
        labelErrs['mtCat'] = mtCat
        labelErrs['lBias'] = lBias
        labelErrs['rBias'] = rBias
        biasDF['bias'] = lBias + rBias
        biasDF['region'] = mtCat + mtCat
        biasDF['img'] = lImg + rImg
        biasDF['urb'] = lUrb + rUrb
        return([labelErrs,biasDF])    
    
    def normalizeVals(self,inputVals):
        minVal = min(inputVals)
        maxVal = max(inputVals)
        bottom = maxVal-minVal
        returnVals = []
        for val in inputVals:
            returnVals.append((val-minVal)/bottom)
        return returnVals
    
    def getCorMatrix(self,postfix):
        labels = ["greenspace_" + postfix,"beauty_" + postfix,"relax_" + postfix,"safe_" + postfix]
        corDict = None
        for label in labels:
            tempDict = deepcopy(self.biasDict[label])
            tempDict = tempDict[[
                'n_' + label[0:-4],
                'avg_mu_' + label[0:-4],
                'avg_sigma_' + label[0:-4],
                'abs_mu_' + label[0:-4],
                'bMean',
                'bStd',
                'image_id'
            ]]
            tempDict.columns = [
                'n_' + label[0:-4],
                'avg_mu_' + label[0:-4],
                'avg_sigma_' + label[0:-4],
                'abs_mu_' + label[0:-4],
                'bias_' + label[0:-4],
                'std_' + label[0:-4],
                'image_id']
            if(labels.index(label) ==0):
                corDict = tempDict
            else:
                corDict = ps.merge(corDict, tempDict, how='inner', on=['image_id'])
        return(corDict)
    
    def getCorPlot(self,postfix,values=True,saveFilename=""): 
        corDict = self.getCorMatrix(postfix)
        labels = ['greenspace','beauty','relax','safe']
        orderedDict = ps.DataFrame()
        for label in labels:
            orderedDict['std_' + label] = corDict['std_' + label]
        for label in labels:
            orderedDict['$\sigma$ ' + label] = corDict['avg_sigma_' + label]
        for label in labels:
            orderedDict['bias ' + label] = corDict['bias_' + label]
        for label in labels:
            orderedDict['$\mu$ ' + label] = corDict['avg_mu_' + label]
        for label in labels:
            orderedDict['intensity ' + label] = corDict['abs_mu_' + label]
        
        corMatrix = orderedDict.corr()
        # Generate the annotation
        annot = np.asarray(corMatrix)
        annot = np.round(annot,2)
        annot[abs(annot) <0.1] = 0.
        annot = annot.astype('str')
        annot[annot=='0.0']=''

        fig, ax = plt.subplots(figsize=(25,15)) 
        if(values):
            sn.heatmap(corMatrix,cmap="PRGn",annot=annot, fmt='')
        else:
            sn.heatmap(corMatrix,cmap="PRGn",fmt='')
        if(len(saveFilename)>0):
            plt.savefig(saveFilename)
        plt.show()
        return(corDict)
    
    def calcAvgImgBias(self,dF):
        biasMean,biasStd,imgArr,mtCat,urbCat,n = [[] for _ in range(6)]
        uniqueImages = list(set(dF['img']))
        for img in uniqueImages:
            imgSubset = dF[dF['img']==img]
            biasMean.append(float(imgSubset['bias'].mean()))
            biasStd.append(float(imgSubset['bias'].std()))
            imgArr.append(img)
            mtCat.append(imgSubset['region'].iloc[0])
            urbCat.append(int(imgSubset['urb'].iloc[0]))
            n.append(len(imgSubset['bias']))
        biasDF = ps.DataFrame()
        biasDF['bMean'] = biasMean
        biasDF['bStd'] = biasStd
        biasDF['img'] = imgArr
        return(biasDF)
    
    def calcAvgImgBiasByRegionOneLabel(self,label,absVal=True):
        err,Bias = self.calcErrsForLabel(label,9000,absVal)
        biasDF = self.calcAvgImgBias(Bias)
        tempScores = self.tsaAbs.getScores(label) if absVal else self.tsaRel.getScores(label)
        tempScores = tempScores[[
            'avg_mu_' + label,
            'avg_sigma_' + label,
            'abs_mu_' + label,
            'image_id',
            'n_' + label  
        ]]
        biasDF = biasDF.merge(tempScores,left_on='img',right_on='image_id',how='inner')
        return(biasDF)
    
    def getDescriptiveStatsOneViewingAngle(self,label,viewCat,absVal='rel',includeLabel=False):
        catName = label + viewCat if includeLabel else viewCat
        angleCat = self.tsaRel.getScores('safe')[['image_id','angle']]
        curDict = deepcopy(self.biasDict[label + "_" + absVal])
        curDict = curDict.merge(angleCat,left_on='image_id',right_on='image_id',how='inner')
        curDict = curDict[curDict['angle']==viewCat]
        curDict = curDict[[
            'n_' + label,
            'avg_mu_' + label,
            'avg_sigma_'+label,
            'bMean',
            'bStd'
        ]]
        curDict.columns = [
            'n_' + catName, 
            'avg_mu_' + catName, 
            'avg_sigma_' + catName,
            'bMean_' + catName,
            'bStd_' + catName
        ]
        return curDict.describe()
    
    def getDescriptiveStatsAllLabels(self,absVal = "rel"):
        descriptiveStatsArray = []
        for label in gConst.OUTCOME_LABELS:
            curDict = deepcopy(self.biasDict[label + "_" + absVal])
            descriptiveStatsArray.append(curDict.describe())
        return(descriptiveStatsArray)
    
    def getDescriptiveStatsByViewingAngleOneLabel(self,label,includeLabel=False):
        descrStats = self.getDescriptiveStatsOneViewingAngle(label,self.VIEWING_ANGLES[0],includeLabel=includeLabel)
        tempStats = self.getDescriptiveStatsOneViewingAngle(label,self.VIEWING_ANGLES[1],includeLabel=includeLabel)
        descrStats = ps.merge(descrStats,tempStats,left_index=True,right_index=True)
        return(descrStats)
    
    def getDescrStatsByViewingAngle(self,includeLabel=False):
        viewingCatStats = {}
        for label in gConst.OUTCOME_LABELS:
            viewingCatStats[label] = self.getDescriptiveStatsByViewingAngleOneLabel(label,includeLabel)
        return(viewingCatStats)
    
    def getDescrStatsOneUrbanCat(self,label,urbanCat,absVal='rel',includeLabel=False):
        catName = label + self.URBAN_MAP[urbanCat] if includeLabel else self.URBAN_MAP[urbanCat] 
        urbanDict = self.tsaRel.getScores('safe')[['image_id','urban_cat']]
        curDict = deepcopy((self.biasDict[label + "_" + absVal]))
        curDict = curDict.merge(urbanDict,left_on="image_id",right_on="image_id",how="inner")
        curDict = curDict[curDict['urban_cat']==urbanCat]
        curDict = curDict[[
            'n_' + label,
            'avg_mu_' + label,
            'avg_sigma_' + label,
            'bMean',
            'bStd'
        ]]
        curDict.columns = [
            'n_'+ catName,
            'avg_mu_' + catName,
            'avg_sigma_' + catName,
            'bMean' + catName,
            'bStd' + catName
        ]
        return curDict.describe()
    
    def getDescrStatsByUrbanCatOneLabel(self,label,includeLabel=False):
        descrStats = self.getDescrStatsOneUrbanCat(label,0,includeLabel=includeLabel)
        tempStats = self.getDescrStatsOneUrbanCat(label,1,includeLabel=includeLabel)
        descrStats = ps.merge(descrStats,tempStats,left_index=True,right_index=True)
        tempStats = self.getDescrStatsOneUrbanCat(label,2,includeLabel=includeLabel)
        descrStats = ps.merge(descrStats,tempStats,left_index=True,right_index=True)
        return(descrStats)
    
    def getDescrStatsByUrbanCat(self,absVal ="rel",includeLabel=False):
        urbanCatStats = {}
        for label in gConst.OUTCOME_LABELS:
            urbanCatStats[label] = self.getDescrStatsByUrbanCatOneLabel(label,includeLabel)
        return(urbanCatStats)
    
    def getBiasDictKeys(self):
        return self.biasDict.keys()
    
    def getBiasDict(self):
        return self.biasDict
    
    def getBiasDictOneLabel(self,label):
        return self.biasDict[label]
    
    def createPlot(self,axs,label,curDict,fig,titleAdj):
        normX = self.normalizeVals((curDict['avg_mu_' + label]))
        normY = self.normalizeVals((curDict['bStd']))
        curPlot = axs.scatter(
            normX,
            normY,
            s=curDict['n_' +label]/2,
            c=curDict['bMean'],
            alpha=1.0,
            label=label
        )
        curPlot.set_clim(-15,15)
        axs.title.set_text(titleAdj + " " + label)
        axs.title.set_fontsize(16)
        divider = make_axes_locatable(axs)
        cax = divider.append_axes('right',size='5%',pad=0.05)
        axs.set_xlim(0,1)
        axs.set_ylim(0,1.2)
        axs.set_xlabel(r'normalized $\mu$',fontsize=14)
        axs.set_ylabel(r'std deviation',fontsize=14)
        axs.tick_params(axis='both',which='major',labelsize=12)
        curPlot.set_cmap("cool")
        fig.colorbar(curPlot,cax=cax,orientation='vertical',label='mean bias')
        l1 = plt.scatter([],[], s=5/2, edgecolors='none',color='gray')
        l2 = plt.scatter([],[], s=10/2, edgecolors='none',color='gray')
        l3 = plt.scatter([],[], s=20/2, edgecolors='none',color='gray')

        labels = ["5", "10", "20"]
        leg = plt.legend([l1, l2, l3], labels, ncol=3, frameon=True, fontsize=10,
        handlelength=2, bbox_to_anchor=(-11.,1), borderpad = 1,
        handletextpad=1, title='n votes', scatterpoints = 1)
    
    def plotSumstats(self,saveFile=""):
        fig, axs = plt.subplots(4,2,figsize=(24,28),dpi=100)
        colIndex,rowIndex = 0,0
        labels = gConst.OUTCOME_LABELS
        for label in labels:
            colIndex=0
            curDict = self.biasDict[label + "_abs"]
            curAxs = axs[rowIndex,colIndex]
            self.createPlot(curAxs,label,curDict,fig,"abs")
            colIndex +=1
            curAxs = axs[rowIndex,colIndex]
            curDict = self.biasDict[label + "_rel"]
            self.createPlot(curAxs,label,curDict,fig,"rel")
            rowIndex+=1
        if(len(saveFile)>0):
            plt.savefig(saveFile)
        plt.show()
        
    def getTSMovingAvgScores(self,absVal=False,runningSize=500):
        self.meanAvgDict = {}
        cleanRecords = self.recordAnalyzer.getCleanRecords().sort_values(by="mt_submit")
        for label in gConst.OUTCOME_LABELS:
            scores = self.tsaAbs.getScores(label) if absVal else self.tsaRel.getScores(label)
            meanArr,curBatchDiff = [],[]
            for index in range(len(cleanRecords['mt_submit'])):
                lImg = list(cleanRecords['l_img_'+label])[index]
                rImg = list(cleanRecords['r_img_'+label])[index]
                lScore = float(scores[scores['image_id']==lImg]['avg_mu_'+label])
                rScore = float(scores[scores['image_id']==rImg]['avg_mu_'+label])
                curBatchDiff.append(abs(lScore-rScore))
                meanArr.append(np.mean(curBatchDiff[max(0,index-runningSize):]))
            self.meanAvgDict[label] = meanArr
        self.meanAvgDict['n'] = len(meanArr)
        print("completed calculating moving avg scores")
        
    def plotTSScores(self,saveFile=False,absVal=False,newGraph=True,runningSize=500):
        if(newGraph): 
            self.getTSMovingAvgScores(absVal,runningSize)
        xVals = list(range(self.meanAvgDict['n']))
        plt.subplots(figsize=(15,10)) 
        for label in gConst.OUTCOME_LABELS:
            plt.errorbar(xVals, self.meanAvgDict[label],label=label)
        plt.legend(loc='lower right')
        plt.title("Moving average TrueSkill scores over time")
        plt.xlabel('record (n)')
        plt.ylabel('TrueSkill score')
        plt.ylim(2.0,7)
        if(saveFile):
            plt.savefig(gConst.MOVING_AVG_TS_FIG)
        plt.show()
        
    def getIntensityMovingAvgScores(self,absVal=False,runningAvgSize=500):
        self.MvIntensityDict = {}
        cleanRecords = self.recordAnalyzer.getCleanRecords().sort_values(by="mt_submit")
        prefix = "abs_cont_outcome_" if absVal else "rel_cont_outcome_"
        for label in gConst.OUTCOME_LABELS:
            meanArr,curBatchDiff = [],[]
            for index in range(len(cleanRecords['mt_submit'])):
                lImg = list(cleanRecords['l_img_'+label])[index]
                rImg = list(cleanRecords['r_img_'+label])[index]
                vote = list(cleanRecords[prefix + label])[index]
                curBatchDiff.append(abs(vote-50)) # intensity of the vote
                meanArr.append(np.mean(curBatchDiff[max(0,len(curBatchDiff)-runningAvgSize):]))
            self.MvIntensityDict[label] = meanArr
        self.MvIntensityDict['n'] = len(meanArr)
        print("completed calculating moving avg scores")
        
    def plotIntensityScores(self,saveFile=False,absVal=False,newGraph=True,runningSize=500):
        if(newGraph): 
            self.getIntensityMovingAvgScores(absVal)
        xVals = list(range(self.MvIntensityDict['n']))
        plt.subplots(figsize=(15,10)) 
        for label in gConst.OUTCOME_LABELS:
            plt.errorbar(xVals, self.MvIntensityDict[lcalcAvgImgBiasByRegionAllLabelsabel],label=label)
        plt.legend(loc='lower right')
        plt.title("Moving average intensity scores over time")
        plt.xlabel('record (n)')
        plt.ylabel('TrueSkill score')
        plt.ylim(20,30)
        if(saveFile):
            plt.savefig(gConst.MOVING_AVG_INTENSITY_FIG)
        plt.show()     
        
    def calcVoterIntensity2(self):
        cleanRecords = self.recordAnalyzer.getCleanRecords().sort_values(by="mt_submit")
        sumIntensity = [0]*len(cleanRecords['mt_submit'])
        prefix = "abs_cont_outcome_" 
        for label in gConst.OUTCOME_LABELS:
            scores = self.tsaAbs.getScores(label) 
            for index in range(len(cleanRecords['mt_submit'])):
                lImg = list(cleanRecords['l_img_'+label])[index]
                rImg = list(cleanRecords['r_img_'+label])[index]
                lScore = float(scores[scores['image_id']==lImg]['avg_mu_'+label])
                rScore = float(scores[scores['image_id']==rImg]['avg_mu_'+label])
                tsDiff = abs(lScore-rScore)
                voteIntensity = abs(list(cleanRecords[prefix + label])[index]-50)
                sumIntensity[index] += (voteIntensity+1)/(tsDiff+1)
        
        self.intensityAdjust = ps.DataFrame()
        self.intensityAdjust['sumIntensity'] = sumIntensity
        self.intensityAdjust['sumIntensity'] = self.intensityAdjust['sumIntensity']/len(gConst.OUTCOME_LABELS)
        maxIntensity = self.intensityAdjust['sumIntensity'].quantile(0.95)
        minIntensity = min(self.intensityAdjust['sumIntensity'])
        rangeInt = maxIntensity-minIntensity
        
        output = []
        for val in self.intensityAdjust['sumIntensity']:
            output.append(1.5 - min(((val - minIntensity)/rangeInt),1))
        self.intensityAdjust['adjVal'] = output

    def createNormAdjust(self):
        print("creating adjusted measure")
        cleanRecords = self.recordAnalyzer.getCleanRecords().sort_values(by="mt_submit")
        prefix = "abs_cont_outcome_" 
        self.calcVoterIntensity2()
        normNorm = []
        for recordIndex in range(len(cleanRecords['mt_submit'])):
            tempVal = self.intensityAdjust['adjVal'].iloc[recordIndex]*cleanRecords['normAdjust'].iloc[recordIndex]
            carAdjust = min(max(0.5,1.5-cleanRecords['normCar'].iloc[recordIndex]),1.5)
            adjVal = (tempVal*0.75+carAdjust*0.25) if tempVal > carAdjust else (tempVal*0.25+carAdjust*0.75)
            normNorm.append(adjVal)
        cleanRecords['normAdjust2'] = normNorm
        self.recordAnalyzer = mTurkAnalyzer.MTurkAnalyzer(timeCutoff=30)
        self.recordAnalyzer.setRecords(cleanRecords)
        self.recordAnalyzer.saveRecordsToCSV(gConst.MTURK_RECORDS_CLEANED)  
    
    def plotNormIntensityScores(self,saveFile="",absVal=False,newGraph=True,runningSize=500):
        if(newGraph): 
            self.getIntesityNormalizedByTS(absVal,runningSize)
        xVals = list(range(self.NormalizedIntensityDict['n']))
        plt.subplots(figsize=(15,10)) 
        
        for label in gConst.OUTCOME_LABELS:
            print(len(self.NormalizedIntensityDict[label]))
            plt.errorbar(
                list(range(len(self.NormalizedIntensityDict[label]))),
                self.NormalizedIntensityDict[label],label=label
            )
        plt.legend(loc='lower right')
        plt.title("Moving average intensity scores over time")
        plt.xlabel('record (n)')
        plt.ylabel('TrueSkill score')
        if(saveFile):
            plt.savefig(gConst.MOVING_AVG_INTENSITY_FIG)
        plt.show()
        
    def getMergedBiasDict(self):
        label = gConst.OUTCOME_LABELS[0]
        baseDict = self.biasDict[label + "_rel"]
        gisData = self.tsaRel.getScores('safe')
        gisData = gisData[['urban_cat','lat','lon','angle','image_id']]
        baseDict = baseDict.merge(gisData,left_on='img',right_on='image_id')
        baseDict['bMean_' + label ] = baseDict['bMean']
        baseDict['bStd_' + label] = baseDict['bStd']
        baseDict = baseDict.drop(columns=['bMean','bStd'])
        for label in gConst.OUTCOME_LABELS[1:]:
            tempDict = self.biasDict[label + "_rel"]
            tempDict['bMean_' + label] = tempDict['bMean']
            tempDict['bStd_' + label] = tempDict['bStd']
            tempDict.drop(columns=['bMean','bStd'])
            baseDict = baseDict.merge(tempDict,left_on="img",right_on="img",how='inner')
        return(baseDict)
    
        # whether to include label in output descriptive stats dictionary
    def getDescriptiveStatsByUrbanCat(self,includeLabel=False):
        urbanCatStats = {}
        for label in gConst.OUTCOME_LABELS:
            dictLabel = label + "_rel" 
            urbanCatStats[label] = self.getDescriptiveStatsByUrbanCatOneLabel(dictLabel,includeLabel)
        return urbanCatStats
      
    def calcAvgImgBiasByRegionAllLabels(self):
        for label in gConst.OUTCOME_LABELS:
            dictLabel = label + "_abs" 
            self.biasDict[dictLabel] = self.calcAvgImgBiasByRegionOneLabel(label,True)
            dictLabel = label + "_rel"
            self.biasDict[dictLabel] = self.calcAvgImgBiasByRegionOneLabel(label,False)
            print("completed calculating bias for label: %s" % (label))